In [ ]:
### Mount google drive if available
try:
    from google.colab import drive
    drive.mount('/content/drive')
    drive_path = '/content/drive/MyDrive/master_thesis/'
    in_colab = True
except:
    drive_path = ''
    in_colab = False

In [ ]:
### Install all dependecies

# pytorch3d
import os
import sys
import torch

need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True

if need_pytorch3d:
    if torch.__version__.startswith("1.10.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'


# smplx
need_smplx=False
try:
    import smplx
except ModuleNotFoundError:
    need_smplx=True

if need_smplx:
    !pip install smplx


# cleanup
!rm -rf 1.10.0.tar.gz cub-1.10.0/

Please upload the `utils` directory and comment out line 7 in `__init__.py`


In [ ]:
try:
    __IPYTHON__
    from tqdm.notebook import tqdm
except NameError:
    from tqdm import tqdm

import os
import cv2
import torch
import smplx
import numpy as np
from torchvision.io import read_image

from pytorch3d.io import load_obj
from pytorch3d.structures import Meshes, Pointclouds
from pytorch3d.transforms import axis_angle_to_matrix
from pytorch3d.renderer import (
    PerspectiveCameras,
    TexturesVertex,
    TexturesUV,
    Materials,
    PointsRasterizationSettings,
    PointsRenderer,
    PointsRasterizer,
    AlphaCompositor,
    NormWeightedCompositor
)

from utils.download_humbi import download_subject, remove_subject
from utils.smpl_to_smplx import humbi_smpl_mesh
from utils.camera_calibration import get_camera_parameters
from utils.renderers import get_renderers

In [ ]:
### Setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
### Load SMPLX mesh from neural rendering output
def load_smplx_mesh(smplx_npz_path, smplx_model_path, smplx_uv_path, smplx_texture_path):
    # load smplx parameters stored in npz file
    global_orient = torch.Tensor(np.load(smplx_npz_path)['global_orient']).to(device)
    transl = torch.Tensor(np.load(smplx_npz_path)['transl']).to(device)
    body_pose = torch.Tensor(np.load(smplx_npz_path)['body_pose']).to(device)
    left_hand_pose = torch.Tensor(np.load(smplx_npz_path)['left_hand_pose']).to(device)
    right_hand_pose = torch.Tensor(np.load(smplx_npz_path)['right_hand_pose']).to(device)
    jaw_pose = torch.Tensor(np.load(smplx_npz_path)['jaw_pose']).to(device)
    expression = torch.Tensor(np.load(smplx_npz_path)['expression']).to(device)
    betas = torch.Tensor(np.load(smplx_npz_path)['betas']).to(device)
    scale = torch.Tensor(np.load(smplx_npz_path)['scale']).to(device)
    verts_disps = torch.Tensor(np.load(smplx_npz_path)['verts_disps']).to(device)

    # construct smplx model
    smplx_model = smplx.SMPLXLayer(smplx_model_path, gender='neutral').to(device)
    smplx_faces = smplx_model.faces_tensor.unsqueeze(0).to(device)
    smplx_verts = smplx_model.forward(global_orient=axis_angle_to_matrix(global_orient),
                                     body_pose=axis_angle_to_matrix(body_pose),
                                     left_hand_pose=axis_angle_to_matrix(left_hand_pose), right_hand_pose=axis_angle_to_matrix(right_hand_pose),
                                     jaw_pose=axis_angle_to_matrix(jaw_pose), expression=expression, betas=betas)['vertices'].to(device)

    # load smplx uv coordinates
    obj_mesh = load_obj(smplx_uv_path, load_textures=False)
    faces_uvs = obj_mesh[1].textures_idx.unsqueeze(0).to(device)
    verts_uvs = obj_mesh[2].verts_uvs.unsqueeze(0).to(device)

    # load smplx texture
    texture = torch.moveaxis(read_image(smplx_texture_path).unsqueeze(0), 1, 3).to(device).float() / 255.0
    texture_uv = TexturesUV(maps=texture, faces_uvs=faces_uvs, verts_uvs=verts_uvs)

    # smplx mesh
    smplx_mesh = Meshes(smplx_verts * scale + transl, smplx_faces, texture_uv)
    smplx_verts = (smplx_verts * scale) + (smplx_mesh.verts_normals_packed() * verts_disps).unsqueeze(0)
    smplx_mesh = Meshes(smplx_verts + transl, smplx_faces, texture_uv)

    return smplx_mesh

In [ ]:
### Get pointcloud renderer
def get_pcl_renderer(cameras, image_size=(360, 640)):
    raster_settings = PointsRasterizationSettings(
        image_size=image_size, 
        radius=0.003,
        points_per_pixel=10
    )

    rasterizer = PointsRasterizer(cameras=cameras, raster_settings=raster_settings)

    pcl_renderer = PointsRenderer(
        rasterizer=rasterizer,
        compositor=AlphaCompositor()
    )

    return pcl_renderer

In [ ]:
SUBJECT = 1

# body
!wget https://humbi-dataset.s3.amazonaws.com/body_subject/subject_$'{SUBJECT}'.zip
!unzip subject_$'{SUBJECT}'.zip
!rm subject_$'{SUBJECT}'.zip

# texture
!wget https://humbi-dataset.s3.amazonaws.com/body_texture_subject/subject_$'{SUBJECT}'.zip
!unzip subject_$'{SUBJECT}'.zip
!rm subject_$'{SUBJECT}'.zip

# point cloud
if SUBJECT == 1:
    !wget 'https://humbi-dataset.s3.amazonaws.com/pointcloud/sample.zip'
    !unzip sample.zip
    !rm sample.zip

In [ ]:
### Render subject in given pose and specified camera view
POSE = '00000025'
CAMERA_IDX = 2

pcl_path = 'subject_%d/body/%s/reconstruction/surface_reconstruction.txt' % (SUBJECT, POSE)

smpl_obj_path = drive_path + 'smplx/smpl_uv.obj'
smpl_texture_path = 'subject_%d/body/%s/appearance/median_map.png' % (SUBJECT, POSE)

smplx_model_path = drive_path + 'smplx'
smplx_obj_path = drive_path + 'smplx/smplx_uv.obj'
smplx_texture_path = drive_path + 'humbi_output/humbi_smplx_rgb/rgb_texture_%d.png' % SUBJECT
smplx_npz_path = drive_path + 'humbi_output/humbi_smplx_npz/output_subject_%d.npz' % SUBJECT

R, T, f, p = get_camera_parameters(SUBJECT, CAMERA_IDX)

# construct camera
cameras = PerspectiveCameras(focal_length=-f, principal_point=p, R=R, T=T, in_ndc=False, image_size=((1080, 1920),), device=device)

# pointcloud
pcl_data = torch.Tensor( np.loadtxt(pcl_path)).unsqueeze(0).to(device)
pcl_data[0,:,-1] = 255 # set all alpha values to full opacity
pointcloud = Pointclouds(points=pcl_data[:,:,:3], features=pcl_data[:,:,3:]/255)

# meshes
smpl_default_mesh = humbi_smpl_mesh(SUBJECT, POSE)

obj_mesh = load_obj(smpl_obj_path, load_textures=False)
faces_uvs = obj_mesh[1].textures_idx.unsqueeze(0).to(device)
verts_uvs = obj_mesh[2].verts_uvs.unsqueeze(0).to(device)
texture = torch.moveaxis(read_image(smpl_texture_path).unsqueeze(0), 1, 3).to(device).float() / 255.0
texture_uv = TexturesUV(maps=texture, faces_uvs=faces_uvs, verts_uvs=verts_uvs)

blank_verts_colors = torch.ones_like(smpl_default_mesh.verts_padded()).to(device)
blank_texture = TexturesVertex(verts_features=blank_verts_colors)

smpl_default_mesh = Meshes(smpl_default_mesh.verts_padded(), smpl_default_mesh.faces_padded(), blank_texture)
smpl_textured_mesh = Meshes(smpl_default_mesh.verts_padded(), smpl_default_mesh.faces_padded(), texture_uv)

smplx_mesh = load_smplx_mesh(smplx_npz_path, smplx_model_path, smplx_obj_path, smplx_texture_path)

# render meshes from camera viewpoint
_, phong_renderer = get_renderers(cameras, (1080, 1920), device=device)
_, phong_renderer_pt_light = get_renderers(cameras, (1080, 1920), pointlight=True, device=device)
pcl_renderer = get_pcl_renderer(cameras, (1080, 1920))

materials = Materials(device=device, specular_color=[[0.0, 0.0, 0.0]], shininess=0.0)
smpl_default_render = phong_renderer_pt_light(smpl_default_mesh, materials=materials)[0].cpu().numpy()
smpl_textured_render = phong_renderer(smpl_textured_mesh)[0].cpu().numpy()
smplx_render = phong_renderer(smplx_mesh)[0].cpu().numpy()
pcl_render = pcl_renderer(pointcloud, device=device)[0].cpu().numpy()

smpl_default_render = (smpl_default_render * 255).astype(np.uint8)
smpl_textured_render = (smpl_textured_render * 255).astype(np.uint8)
smplx_render = (smplx_render * 255).astype(np.uint8)
pcl_render = (pcl_render * 255).astype(np.uint8)

# store images
cam_idx_str = '%07d' % CAMERA_IDX
savefile_smpl_default = 'smpl_default_%s.png' % cam_idx_str
savefile_smpl_textured = 'smpl_textured_%s.png' % cam_idx_str
savefile_smplx_textured = 'smplx_textured_%s.png' % cam_idx_str
savefile_pointcloud = 'pointcloud_%s.png' % cam_idx_str

success = cv2.imwrite(savefile_smpl_default, cv2.cvtColor(smpl_default_render, cv2.COLOR_BGRA2RGBA))
success = cv2.imwrite(savefile_smpl_textured, cv2.cvtColor(smpl_textured_render, cv2.COLOR_BGRA2RGBA))
success = cv2.imwrite(savefile_smplx_textured, cv2.cvtColor(smplx_render, cv2.COLOR_BGRA2RGBA))
success = cv2.imwrite(savefile_pointcloud, cv2.cvtColor(pcl_render, cv2.COLOR_BGRA2RGBA))